# Final Deliverable

## Business Problem

I am an immigrant in Canada from India with a cooking expertise. I want to open a food-chain focusing on Indian cuisine in Toronto. For the starting of the business, I need some suggestion regarding the neighborhood of Toronto. In this regard, I have two options: being more competitive or being for innovative. If I want to go competitive then I want to locate my first restaurant to be in the center where there are a lot of highly rated Indian restaurant. Else I want to locate my restaurant somewhere nearby where Indian cuisines are not available or not that much highly rated. I need both kind of suggestion in terms of location.

## Data

We will be using Foursquare API to explore a geographical location. We will be using this to locate Indian restaurants. We will also ne using Foursquare API for the user rating of those places. And for the latitude and longitude we will be using the file provided in http://cocl.us/Geospatial_data.

### Importing necessary library

In [119]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import numpy as np
from geopy.geocoders import Nominatim
import folium

### User Credentials

In [3]:
CLIENT_ID = 'MFJLBG2F0TYVVTI3DHZASIBPL2AUB5AW3GCMSY4ZZR0UTL4U' # your Foursquare ID
CLIENT_SECRET = 'OVPAA3IQLB45V3LRO1H0TMFP1RBUMV4XFPC40DHC2MALLI5X' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MFJLBG2F0TYVVTI3DHZASIBPL2AUB5AW3GCMSY4ZZR0UTL4U
CLIENT_SECRET:OVPAA3IQLB45V3LRO1H0TMFP1RBUMV4XFPC40DHC2MALLI5X


### Loading location data with postal codes in Toronto

In [4]:
location_data = pd.read_csv("http://cocl.us/Geospatial_data")
location_data.columns=['PostalCode','Latitude','Longitude']
location_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Search for first geographical coordinate for "Indian" cuisine in 3Km radius

In [19]:
search_query = 'Indian'
radius = 3000

In [156]:
result=pd.DataFrame(columns=['name','id','location.lat','location.lng','categories'])
for lat,long in zip(location_data['Latitude'],location_data['Longitude']):
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    #dataframe['categories']=dataframe['categories'].apply(lambda x: x[0]['name'])
    dataframe=dataframe[['name','id','location.lat','location.lng','categories']]
    result=pd.concat([result,dataframe],axis=0)
result.head()

,name,id,location.lat,location.lng,categories
0,Charley's West Indian Foods Ltd,4d2c8332342d6dcbe82020cb,43.800581,-79.199309,"[{'id': '4bf58dd8d48988d144941735', 'name': 'C..."
1,Indian Rhinoceros Pavilion,4cd8637d145fa1cdda2b06bd,43.819266,-79.181535,"[{'id': '58daa1558bbb0b01f18ec1fd', 'name': 'Z..."
2,Rubini West Indian Food Market,4e35b8a8ae60d86c3ac2e8d4,43.807465,-79.221210,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G..."
0,Charley's West Indian Foods Ltd,4d2c8332342d6dcbe82020cb,43.800581,-79.199309,"[{'id': '4bf58dd8d48988d144941735', 'name': 'C..."
0,Harry's West Indian Fine Foods,4f380d00e4b039c3c31e1dd4,43.759580,-79.223818,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G..."


In [157]:
def get_venue_category(x):
    try:
        return x[0]['name']
    except:
        return np.nan

In [158]:
result['categories']=result['categories'].apply(lambda x: get_venue_category(x))

### Data Cleaning

### Dropping the dupliacates

In [159]:
result.drop_duplicates(['name'],inplace=True)
result.head()

,name,id,location.lat,location.lng,categories
0,Charley's West Indian Foods Ltd,4d2c8332342d6dcbe82020cb,43.800581,-79.199309,Caribbean Restaurant
1,Indian Rhinoceros Pavilion,4cd8637d145fa1cdda2b06bd,43.819266,-79.181535,Zoo Exhibit
2,Rubini West Indian Food Market,4e35b8a8ae60d86c3ac2e8d4,43.807465,-79.221210,Grocery Store
0,Harry's West Indian Fine Foods,4f380d00e4b039c3c31e1dd4,43.759580,-79.223818,Grocery Store
1,Markham Danforth West Indian Food Market,5156f7aae4b0e3e0a5c7819f,43.767399,-79.228172,Grocery Store


In [160]:
result.shape

(153, 5)

### Check if there is any null value

In [161]:
result.isnull().sum()

name            0
id              0
location.lat    0
location.lng    0
categories      7
dtype: int64

In [162]:
result.dropna(inplace=True)

In [163]:
result.reset_index(drop=True,inplace=True)
result.tail()

,name,id,location.lat,location.lng,categories
141,GTA & Indian Cuisine,4dcc0169c65bccd86753084f,43.701689,-79.575047,Pizza Place
142,Indian Line Park,4fc15c57e4b0d3da59028b33,43.726021,-79.622084,Park
143,Exotic indian cuisine,4df24f1b88772e1f814f40d3,43.745956,-79.615838,Indian Restaurant
144,Coach Canada,4f9e7613e4b044503c54970e,43.692659,-79.606970,General Travel
145,Food Fiesta,598837924c954c796414b0f3,43.719947,-79.638541,Grocery Store


In [164]:
result.isnull().sum()

name            0
id              0
location.lat    0
location.lng    0
categories      0
dtype: int64

### Visualization

In [165]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [166]:
# create map
map = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to the map
markers_colors = []
for lat, lon, poi in zip(result['location.lat'], result['location.lng'], result['name']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label).add_to(map)
       
map

### Find all restaurants

In [167]:
result['categories'].unique()
words=['Restaurant','Buffet','Food','Bar','Pizza','Chicken']
pattern = '|'.join(words)
restaurant=result.copy()

In [168]:
restaurant=restaurant[restaurant['categories'].str.contains(pattern)]
restaurant.reset_index(drop=True,inplace=True)
restaurant.head()

,name,id,location.lat,location.lng,categories
0,Charley's West Indian Foods Ltd,4d2c8332342d6dcbe82020cb,43.800581,-79.199309,Caribbean Restaurant
1,Vinchandra Fine Indian Cuisine,58e46b06a4367c749a6b149f,43.767282,-79.228109,Restaurant
2,Karaikudi Chettinad South Indian Restaurant,4bf96c435317a593a23a017f,43.756042,-79.276276,Indian Restaurant
3,Tandori Indian Cuisine,52054149498e93821253f4d5,43.775032,-79.257064,Indian Restaurant
4,Silver Spoon Pak-Indian Restaurant,4d570727fb65236a7f600db4,43.791930,-79.251253,Indian Restaurant


In [169]:
restaurant.shape

(95, 5)

In [170]:
# create map
map = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to the map
for lat, lon, poi in zip(result['location.lat'], result['location.lng'], result['name']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color='blue',
        popup=label).add_to(map)
for lat, lon, poi in zip(restaurant['location.lat'], restaurant['location.lng'], restaurant['name']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color='red',
        popup=label).add_to(map)      
map

### Gettting the venue rating

In [146]:
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4fc15c57e4b0d3da59028b33,NaN,CA,NaN,Canada,NaN,3112,[Canada],"[{'label': 'display', 'lat': 43.7260213949842,...",43.726021,-79.622084,NaN,NaN,Indian Line Park,v-1591962728
1,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,4dcc0169c65bccd86753084f,NaN,CA,Toronto,Canada,NaN,1629,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.701689, 'lng':...",43.701689,-79.575047,NaN,ON,GTA & Indian Cuisine,v-1591962728
2,"[{'id': '4bf58dd8d48988d1f6931735', 'name': 'G...",False,4f9e7613e4b044503c54970e,6020 Indian Line,CA,Mississauga,Canada,NaN,1881,"[6020 Indian Line, Mississauga ON L4V 1G5, Can...","[{'label': 'display', 'lat': 43.692659135308, ...",43.692659,-79.606970,L4V 1G5,ON,Coach Canada,v-1591962728
3,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4e6155c97d8b8540892c9de7,2257 Islington Ave.,CA,Etobicoke,Canada,2257 Isllington Avenue,3285,"[2257 Islington Ave. (2257 Isllington Avenue),...","[{'label': 'display', 'lat': 43.71612192694068...",43.716122,-79.555331,NaN,ON,Family Indian Cuisine,v-1591962728
4,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,598837924c954c796414b0f3,7205 Goreway Dr,CA,Mississauga,Canada,NaN,3869,"[7205 Goreway Dr, Mississauga ON L4T 2T9, Canada]","[{'label': 'display', 'lat': 43.71994692317504...",43.719947,-79.638541,L4T 2T9,ON,Food Fiesta,v-1591962728


## Methodology

In [175]:
points=result[['location.lat','location.lng']].values

In [180]:
# clustering

from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import math


def distance(origin, destination):
    lat1, lon1 = origin[0],origin[1]
    lat2, lon2 = destination[0],destination[1]
    radius = 6371 # km
    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

def create_clusters(number_of_clusters,points):
    kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(points)
    l_array = np.array([[label] for label in kmeans.labels_])
    clusters = np.append(points,l_array,axis=1)
    return clusters

def validate_solution(max_dist,clusters):
    _, __, n_clust = clusters.max(axis=0)
    n_clust = int(n_clust)
    for i in range(n_clust):
        two_d_cluster=clusters[clusters[:,2] == i][:,np.array([True, True, False])]
        if not validate_cluster(max_dist,two_d_cluster):
            return False
        else:
            continue
    return True

def validate_cluster(max_dist,cluster):
    distances = cdist(cluster,cluster, lambda ori,des: int(round(distance(ori,des))))
    #print(distances)
    #print(30*'-')
    for item in distances.flatten():
        if item > max_dist:
            return False
    
    return True

for i in range(2,len(points)):
    print(i)
    print(validate_solution(5,create_clusters(i,points)))

2
False
3
False
4
False
5
False
6
False
7
False
8
False
9
False
10
False
11
False
12
False
13
False
14
False
15
False
16
False
17
False
18
False
19
False
20
False
21
False
22
False
23
False
24
False
25
False
26
True
27
False
28
False
29
False
30
True
31
True
32
True
33
True
34
True
35
False
36
True
37
True
38
True
39
True
40
True
41
True
42
True
43
True
44
True
45
True
46
True
47
True
48
True
49
True
50
True
51
True
52
True
53
True
54
True
55
True
56
True
57
True
58
True
59
True
60
True
61
True
62
True
63
True
64
True
65
True
66
True
67
True
68
True
69
True
70
True
71
True
72
True
73
True
74
True
75
True
76
True
77
True
78
True
79
True
80
True
81
True
82
True
83
True
84
True
85
True
86
True
87
True
88
True
89
True
90
True
91
True
92
True
93
True
94
True
95
True
96
True
97
True
98
True
99
True
100
True
101
True
102
True
103
True
104
True
105
True
106
True
107
True
108
True
109
True
110
True
111
True
112
True
113
True
114
True
115
True
116
True
117
True
118
True
119
True
120
True
121
Tru

In [182]:
# Using the lowest number corresponding to TRUE
number_of_clusters=26
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(points)
l_array = np.array([[label] for label in kmeans.labels_])
clusters = np.append(points,l_array,axis=1)

In [192]:
result['cluster']=(clusters[:,2]).tolist()

In [194]:
result.head()

,name,id,location.lat,location.lng,categories,cluster
0,Charley's West Indian Foods Ltd,4d2c8332342d6dcbe82020cb,43.800581,-79.199309,Caribbean Restaurant,6.0
1,Indian Rhinoceros Pavilion,4cd8637d145fa1cdda2b06bd,43.819266,-79.181535,Zoo Exhibit,6.0
2,Rubini West Indian Food Market,4e35b8a8ae60d86c3ac2e8d4,43.807465,-79.221210,Grocery Store,6.0
3,Harry's West Indian Fine Foods,4f380d00e4b039c3c31e1dd4,43.759580,-79.223818,Grocery Store,14.0
4,Markham Danforth West Indian Food Market,5156f7aae4b0e3e0a5c7819f,43.767399,-79.228172,Grocery Store,14.0


In [203]:
cluster_data=pd.DataFrame(kmeans.cluster_centers_,columns=['latitude','longitude'])
cluster_data['cluster']=np.arange(1,len(cluster_data)+1)
cluster_data.head()

,latitude,longitude,cluster
0,43.694249,-79.506325,1
1,43.758170,-79.279904,2
2,43.655299,-79.383973,3
3,43.703490,-79.579116,4
4,43.653738,-79.462783,5


In [204]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(number_of_clusters)
ys = [i + x + (i*x)**2 for i in range(number_of_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(result['location.lat'], result['location.lng'], result['cluster']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
for lat, lon, cluster_no in zip(cluster_data['latitude'], cluster_data['longitude'], cluster_data['cluster']):
    label = folium.Popup(' Cluster ' + str(cluster_no), parse_html=True)
    folium.Marker(
        [lat, lon],
        radius=8,
        popup=label,
        color='Yelloe',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [206]:
# create clusters for the restaurants
points=restaurant[['location.lat','location.lng']].values
for i in range(2,len(points)):
    print(i)
    print(validate_solution(5,create_clusters(i,points)))

2
False
3
False
4
False
5
False
6
False
7
False
8
False
9
False
10
False
11
False
12
False
13
False
14
False
15
False
16
False
17
False
18
False
19
False
20
True
21
False
22
True
23
True
24
True
25
True
26
True
27
True
28
True
29
True
30
True
31
True
32
True
33
True
34
True
35
True
36
True
37
True
38
True
39
True
40
True
41
True
42
True
43
True
44
True
45
True
46
True
47
True
48
True
49
True
50
True
51
True
52
True
53
True
54
True
55
True
56
True
57
True
58
True
59
True
60
True
61
True
62
True
63
True
64
True
65
True
66
True
67
True
68
True
69
True
70
True
71
True
72
True
73
True
74
True
75
True
76
True
77
True
78
True
79
True
80
True
81
True
82
True
83
True
84
True
85
True
86
True
87
True
88
True
89
True
90
True
91
True
92
True
93
True
94
True


In [207]:
# Using the lowest number corresponding to TRUE
number_of_clusters=20
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(points)
l_array = np.array([[label] for label in kmeans.labels_])
clusters = np.append(points,l_array,axis=1)

In [208]:
restaurant['cluster']=(clusters[:,2]).tolist()
restaurant.head()

,name,id,location.lat,location.lng,categories,cluster
0,Charley's West Indian Foods Ltd,4d2c8332342d6dcbe82020cb,43.800581,-79.199309,Caribbean Restaurant,6.0
1,Vinchandra Fine Indian Cuisine,58e46b06a4367c749a6b149f,43.767282,-79.228109,Restaurant,6.0
2,Karaikudi Chettinad South Indian Restaurant,4bf96c435317a593a23a017f,43.756042,-79.276276,Indian Restaurant,17.0
3,Tandori Indian Cuisine,52054149498e93821253f4d5,43.775032,-79.257064,Indian Restaurant,6.0
4,Silver Spoon Pak-Indian Restaurant,4d570727fb65236a7f600db4,43.791930,-79.251253,Indian Restaurant,6.0


In [209]:
cluster_data2=pd.DataFrame(kmeans.cluster_centers_,columns=['latitude','longitude'])
cluster_data2['cluster']=np.arange(1,len(cluster_data2)+1)
cluster_data2.head()

,latitude,longitude,cluster
0,43.661561,-79.483155,1
1,43.733866,-79.307078,2
2,43.671495,-79.330636,3
3,43.756680,-79.530373,4
4,43.654742,-79.383729,5


In [210]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(number_of_clusters)
ys = [i + x + (i*x)**2 for i in range(number_of_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(restaurant['location.lat'], restaurant['location.lng'], restaurant['cluster']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
for lat, lon, cluster_no in zip(cluster_data2['latitude'], cluster_data2['longitude'], cluster_data2['cluster']):
    label = folium.Popup(' Cluster ' + str(cluster_no), parse_html=True)
    folium.Marker(
        [lat, lon],
        radius=8,
        popup=label,
        color='Yelloe',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Result and Discussions

We can see the some highly densed restaurants in the shore area and mostly the places labeled as ‘Indian’ are restaurants.
After clustering we get 26 clusters for all the Indian places e.g. restaurants, grocery store, religious places etc. And after clustering the restaurants we get 20 clusters of Indian restaurants. By this inspection we can say approximately there are four other places where our restaurant can be set up.From comparing the clusters we get four location data where a restaurant can be set up. But exploring a new area for a restaurant focusing on a foreign cuisine can be little risky as well. So, the answer comes with the client’s preference, whether he will be competitive or innovative. In the first case, he can go for the heavily densed shore area where we already have a tons of Indian restaurants, and there will be less trouble for making crowd. But there will be competition from others. And in the latter case, he will explore a new area where he will have to draw attention of the crowd from the start but there will be no competition from others.


## Conclusions

We can run the same process after we have enough data in the foursquare API with the user rating of those ‘Indian’ place when we will be able to suggest more accurate and demanding area for the restaurant. If our client wants to upgrade to a multi-cuisine restaurant in the future then this process can be re-run for all the restaurants with same cuisine. By this method, we can offer our client with locations suitable to their taste and maximize the profit.